<a href="https://colab.research.google.com/github/AhmedNadir314/alsd-bayesian-optimization/blob/main/building_sampling_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Buildng the sampling frame includes:

1. Dropping households with h_cell = 'no'

2. Dropping households with N_cell = 0

3. Repeating each row N_cell times

4. Creating the final index (IDs)

In [43]:
import pandas as pd

url = "https://raw.githubusercontent.com/AhmedNadir314/alsd-bayesian-optimization/main/data/popframe_dhs_census22_adm1.csv"
df = pd.read_csv(url)
df.head()

,adm1_name,adm2_name,h_water_source_2way,hh_size_bucket,h_cooking_fuel,h_toilet_type_3way,h_toilet_type_2way,hhh_sex,hhh_edu,h_cell,pop_weight,norm_weight,N_cell
0,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,no,0.406199,0.001392,14
1,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,yes,7.212800,0.024709,247
2,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,preprimary/other,yes,0.514982,0.001764,18
3,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,primary,no,0.239905,0.000822,8
4,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,primary,yes,4.699558,0.016099,161


Total number of rows

In [12]:
nrow, ncol = df.shape
print("Number of rows = ", nrow)

Number of rows =  6226


Total number of households

In [13]:
N=df['N_cell'].sum()
print("Total number of households = ", N)

Total number of households =  99992


Households without phones

In [14]:
df[df['h_cell']=='no']['N_cell'].sum()

np.int64(12895)

Dropping households without phones

In [15]:
clean_df=df[df['h_cell']!='no']

Rows with zero N_cell

In [17]:
len(df[df['N_cell']==0])

1496

Dropping rows with zero N_cell

In [18]:
clean_df=clean_df[clean_df['N_cell']!=0]

Define a function to repeat rows

In [38]:
def replicate(data, row, n):
    n = int(n)
    for _ in range(n):
        data.append(row.to_dict())
    return data

creating the sampling frame by repeating rows

In [39]:
s_frame=[]
for _, row in clean_df.iterrows():
  s_frame=replicate(s_frame,row,row['N_cell'])


s_frame = pd.DataFrame(s_frame)
s_frame.drop(columns=['N_cell'])
s_frame['ID']=range(s_frame.shape[0])
s_frame.set_index('ID',inplace=True)

Final sampling frame

In [41]:
s_frame.head()

,adm1_name,adm2_name,h_water_source_2way,hh_size_bucket,h_cooking_fuel,h_toilet_type_3way,h_toilet_type_2way,hhh_sex,hhh_edu,h_cell,pop_weight,norm_weight,N_cell
ID,,,,,,,,,,,,,
0,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,yes,7.2128,0.024709,247
1,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,yes,7.2128,0.024709,247
2,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,yes,7.2128,0.024709,247
3,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,yes,7.2128,0.024709,247
4,Bulawayo,Bulawayo,improved,1-2,clean,improved,improved,F,higher,yes,7.2128,0.024709,247


In [45]:
s_frame.to_csv("sampling_frame.csv", index=False)